# Greedy Agents Decrease Biosphere Lifespan in a DaisyWorld Environment

In [ ]:
import os
import json

import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation
matplotlib.rcParams["animation.embed_limit"] = 256
matplotlib.rcParams["font.size"] = 22

import IPython

import daisy
import daisy.daisy_world_rl
from daisy.daisy_world_rl import RLDaisyWorld
from daisy.agents.greedy import Greedy
from daisy.agents.mlp import MLP


from daisy.daisy_world_simple import SimpleDaisyWorld

#from daisy.agents.greedy import Greedy

# functions for animating daisyworld env
from daisy.notebook_helpers import tensor_to_image,\
        plot_grid,\
        get_update_fig,\
        seed_all
        
my_seed = 42 

In [ ]:
def get_env_and_agent(daisy_status, agent_status, batch_size=1):
    
    env = daisy.daisy_world_rl.RLDaisyWorld(grid_dimension=8)
    env.batch_size = batch_size

    if daisy_status == "neutral_albedo":
        env.albedo_dark = env.albedo_bare
        env.albedo_light = env.albedo_bare

    if agent_status == "greedy":
        agent = Greedy()
        agent.epsilon = 0.0
    elif agent_status == "antigreedy":
        agent = Greedy()
        agent.greedy = False
        agent.epsilon = 0.0
    elif agent_status == "random":
        agent = Greedy()
        agent.epsilon = 1.0
    elif agent_status == "half_random":
        agent = Greedy()
        agent.epsilon = 0.5
    else: 
        agent = None
    
    return env, agent

def simulate_lifespan(env, agent):
    
    steps = []
    for ii in range(1):

        all_done = False
        obs = env.reset()
        done_at = np.zeros((*obs.shape[:1]), dtype=int)
        agents_done_at = np.zeros((*obs.shape[:2],1), dtype=int)

        
        while not all_done:

            if agent is not None:
                action = agent(obs)
            else:
                action = None

            obs, reward, done, info = env.step(action)

            grid_done = env.grid[:,1:3,:,:].max(axis=(1,2,3), keepdims = False) <= 0.005
            done_at += (1 - 1 * grid_done)
            
            agents_done_at += (1 - 1 * done)

            if grid_done.mean() == 1.0:
                all_done = True
        

    return done_at, agents_done_at


In [ ]:
matplotlib.rcParams["font.size"] = 10

seed_all(my_seed)
save_gif = False #True
save_video = False #True

In [ ]:
# Light and dark daisies stabilize the climate of DaisyWorld

In [ ]:
# neutral albedo scenario
# ad = 0.5; al = 0.5; ab = 0.5

daisy_status = "neutral_albedo"
agent_status = "no"

env, agent = get_env_and_agent(daisy_status, agent_status)

In [ ]:
# light and dark daisies scenario
# ad = 0.25; al = 0.75; ab = 0.5

daisy_status = "light_and_dark"
agent_status = "no"

env, agent = get_env_and_agent(daisy_status, agent_status)

In [ ]:
fig, ax = plot_grid(env)

obs = env.reset()
num_frames = env.ramp_period 
interval = 40

update_fig = get_update_fig(env, agent=agent)
  
    
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_fig, \
                    frames=num_frames, interval=interval).to_jshtml())

In [ ]:
# Greedy agents decrease the lifespan of the DaisyWorld biosphere

In [ ]:
matplotlib.rcParams["font.size"] = 8

seed_all(my_seed)
save_gif = False #True
save_video = False #True

In [ ]:
# light and dark daisies scenario, with agents
# ad = 0.25; al = 0.75; ab = 0.5

daisy_status = "light_and_dark"
agent_status = "greedy"

env, agent = get_env_and_agent(daisy_status, agent_status)

In [ ]:
# light and dark daisies scenario, with agents
# ad = 0.25; al = 0.75; ab = 0.5

daisy_status = "light_and_dark"
agent_status = "random"

env, agent = get_env_and_agent(daisy_status, agent_status)

In [ ]:
# light and dark daisies scenario, with agents
# ad = 0.25; al = 0.75; ab = 0.5

daisy_status = "light_and_dark"
agent_status = "antigreedy"

env, agent = get_env_and_agent(daisy_status, agent_status)

In [ ]:
# light and dark daisies scenario, with agents
# ad = 0.25; al = 0.75; ab = 0.5

daisy_status = "light_and_dark"
agent_status = "half_random"

env, agent = get_env_and_agent(daisy_status, agent_status)

In [ ]:
fig, ax = plot_grid(env)

obs = env.reset()
num_frames = env.ramp_period 
interval = 40

update_fig = get_update_fig(env, agent=agent)
        
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_fig, \
                    frames=num_frames, interval=interval).to_jshtml())

In [ ]:
save_gif = False
save_results = True

batch_size = 1000
my_seed = 13

if save_results:
    results = {}

for agent_status in ["greedy", "antigreedy", "random", "half_random", "no"]:
    for daisy_status in ["light_and_dark", "neutral_albedo"]:
        
        seed_all(my_seed)
        env, agent = get_env_and_agent(daisy_status, agent_status, batch_size=batch_size)

        obs = env.reset()
        num_frames = env.ramp_period 
        interval = 40
        
        fig, ax = plot_grid(env)
        update_fig = get_update_fig(env, agent=agent)
        
        if save_video:
            matplotlib.animation.FuncAnimation(fig, update_fig, \
                    frames=num_frames, interval=interval).save(\
                    f"greedy_demo_{daisy_status}_daisies_{agent_status}_agent.mp4")
            
        elif save_gif:
            matplotlib.animation.FuncAnimation(fig, update_fig, \
                    frames=num_frames, interval=interval).save(\
                    f"greedy_demo_{daisy_status}_daisies_{agent_status}_agent.gif")
            
        obs = env.reset()
        
        if save_results:
            key = f"{agent_status}_{daisy_status}"

            results[key] = simulate_lifespan(env, agent)
        

In [ ]:
matplotlib.rcParams["font.size"] = 14
fig, ax = plt.subplots(1,1, figsize=(12,8)) #figure()
my_cmap = plt.get_cmap("viridis")

my_cmap2 = plt.get_cmap("magma")

ticks = []
labels = []

for xx, key in enumerate(results.keys()):
    print("\n", key)
    
    if "light_and_dark" in key:
        mean_env_lifespan = np.mean(results[key][0])
        mean_agent_lifespan = np.mean(results[key][1]) 

        std_dev_env_lifespan = np.std(results[key][0]) / np.sqrt(results[key][0].shape[0])
        std_dev_agent_lifespan = np.std(results[key][1]) / np.sqrt(results[key][0].shape[0])

        env_msg = f"{key} mean lifespan: {mean_env_lifespan:.3f} +/- {std_dev_env_lifespan:.3f}"
        print(env_msg)

        env_color = my_cmap(xx*32)
        agent_color = [1.0 - elem for elem in my_cmap2((2 + xx)*16)[:3]]

        ax.bar(xx, mean_env_lifespan, width=0.25,\
                yerr=std_dev_env_lifespan, capsize=4,\
                label=f"{key} biosphere lifespan", color=env_color, alpha=0.75)

        ticks.append(xx-0.125)

        labels.append(key)

        if "no_" in key:
            print("(no agents)")
        else:
            agent_msg = f"{key} mean agent lifespan: {mean_agent_lifespan:.3f} +/- {std_dev_agent_lifespan:.3f}"
            print(agent_msg)
            ax.bar(xx+0.25, mean_agent_lifespan, width=0.25,\
                    yerr=std_dev_agent_lifespan, capsize=4,\
                    label=f"{key} agent lifespan", color=agent_color, alpha=0.75)

ax.set_xticks(ticks)
ax.set_xticklabels(labels, rotation=80)    
plt.legend(fontsize=10, loc=(1,.25))
ax.set_ylabel("Lifespan (steps) $\\mu \\pm \\frac{\\sigma}{\\sqrt{N}}$")
ax.set_title("Biosphere and Agent Longevity \n DaisyWorld")
ax.axis([-.25, xx+.25, 0, 512])
plt.tight_layout()
plt.savefig("biosphere_longevity.png")
plt.show()

In [ ]:
matplotlib.rcParams["font.size"] = 14
fig, ax = plt.subplots(1,1, figsize=(12,8)) #figure()
my_cmap = plt.get_cmap("viridis")

my_cmap2 = plt.get_cmap("magma")

ticks = []
labels = []

for xx, key in enumerate(results.keys()):
    print("\n", key)
    
    if "light_and_dark" not in key:
        mean_env_lifespan = np.mean(results[key][0])
        mean_agent_lifespan = np.mean(results[key][1]) 

        std_dev_env_lifespan = np.std(results[key][0]) / np.sqrt(results[key][0].shape[0])
        std_dev_agent_lifespan = np.std(results[key][1]) / np.sqrt(results[key][0].shape[0])

        env_msg = f"{key} mean lifespan: {mean_env_lifespan:.3f} +/- {std_dev_env_lifespan:.3f}"
        print(env_msg)

        env_color = my_cmap(xx*32)
        agent_color = [1.0 - elem for elem in my_cmap2((2 + xx)*16)[:3]]

        ax.bar(xx, mean_env_lifespan, width=0.25,\
                yerr=std_dev_env_lifespan, capsize=4,\
                label=f"{key} biosphere lifespan", color=env_color, alpha=0.75)

        ticks.append(xx-0.125)

        labels.append(key)

        if "no_" in key:
            print("(no agents)")
        else:
            agent_msg = f"{key} mean agent lifespan: {mean_agent_lifespan:.3f} +/- {std_dev_agent_lifespan:.3f}"
            print(agent_msg)
            ax.bar(xx+0.25, mean_agent_lifespan, width=0.25,\
                    yerr=std_dev_agent_lifespan, capsize=4,\
                    label=f"{key} agent lifespan", color=agent_color, alpha=0.75)

ax.set_xticks(ticks)
ax.set_xticklabels(labels, rotation=80)    
plt.legend(fontsize=10, loc=(1,.25))
ax.set_ylabel("Lifespan (steps) $\\mu \\pm \\frac{\\sigma}{\\sqrt{N}}$")
ax.set_title("Biosphere and Agent Longevity \n Neutral Albedo DaisyWorld")
ax.axis([-.25, xx+.25, 0, 512])
plt.tight_layout()
plt.savefig("neutral_biosphere_longevity.png")
plt.show()